#**Use Web Scrapping and Cloud API to Perform NLP Analysis on News **








##Scrapping Website to Get News Data

In [0]:
import requests # used to send web-requests (to fetch the html files from websites)
import urllib.request
import time
from bs4 import BeautifulSoup #navigates through the html extract the selected text

In [2]:
index = "https://techcrunch.com/"
requests.get(index) #response 200 means it went through successfully

<Response [200]>

In [0]:
response = requests.get(index)
response.text #raw html text
soup = BeautifulSoup(response.text, "html.parser") #parse the html with BeautifulSoup to get a nested BeautifulSoup data structure.

In [4]:
soup.findAll('a') #get all the links to articles

[<a class="post-block__title__link" href="https://techcrunch.com/2019/03/17/these-are-the-robots-that-help-you-get-your-amazon-packages-on-time/">
 				These are the robots that help you get your Amazon packages on time			</a>,
 <a href="https://techcrunch.com/author/brian-heater/">
 				Brian Heater			</a>,
 <a href="https://techcrunch.com/2019/03/17/these-are-the-robots-that-help-you-get-your-amazon-packages-on-time/">
 <!-- Attaching the URL params manually so that the URL on the images loaded match exactly with React -->
 <img src="https://techcrunch.com/wp-content/uploads/2019/03/amazon_007.jpg?w=300&amp;h=160&amp;crop=1"/>
 </a>,
 <a class="post-block__title__link" href="https://techcrunch.com/2019/03/17/doorport-wants-to-make-your-apartment-buildings-front-door-smarter/">
 				Doorport wants to make your apartment building’s front door smarter			</a>,
 <a href="https://techcrunch.com/author/greg-kumparak/">
 				Greg Kumparak			</a>,
 <a href="https://techcrunch.com/2019/03/17/d

In [18]:
#find the link for the fist article
one_a_tag = soup.findAll('a')[0]
link = one_a_tag['href']
link
#for link in soup.find_all('a'):
#    print(link.get('href'))

'https://techcrunch.com/2019/03/17/doorport-wants-to-make-your-apartment-buildings-front-door-smarter/'

In [0]:
#scrap the article page
response_article = requests.get(link)
article = BeautifulSoup(response_article.text, "html.parser") #parse the html with BeautifulSoup to get a nested BeautifulSoup data structure.

In [20]:
article

<!DOCTYPE html>

<html class="no-js" lang="en-US">
<head>
<meta charset="utf-8"/>
<meta content="IE=Edge" http-equiv="X-UA-Compatible"><script type="text/javascript">window.NREUM||(NREUM={}),__nr_require=function(e,n,t){function r(t){if(!n[t]){var o=n[t]={exports:{}};e[t][0].call(o.exports,function(n){var o=e[t][1][n];return r(o||n)},o,o.exports)}return n[t].exports}if("function"==typeof __nr_require)return __nr_require;for(var o=0;o<t.length;o++)r(t[o]);return r}({1:[function(e,n,t){function r(){}function o(e,n,t){return function(){return i(e,[c.now()].concat(u(arguments)),n?null:this,t),n?void 0:this}}var i=e("handle"),a=e(3),u=e(4),f=e("ee").get("tracer"),c=e("loader"),s=NREUM;"undefined"==typeof window.newrelic&&(newrelic=s);var p=["setPageViewName","setCustomAttribute","setErrorHandler","finished","addToTrace","inlineHit","addRelease"],d="api-",l=d+"ixn-";a(p,function(e,n){s[n]=o(d+n,!0,"api")}),s.addPageAction=o(d+"addPageAction",!0),s.setCurrentRouteName=o(d+"routeName",!0),n.ex

In [0]:
#get the visible text from the article page
#get_text()here will return a bunch of tags, so need to filter them out
from bs4.element import Comment
def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True

def text_from_html(body):
    soup = BeautifulSoup(body, 'html.parser')
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)  
    return u" ".join(t.strip() for t in visible_texts)

html = urllib.request.urlopen(link).read()

In [0]:
#store scrapped texts
data = ""
data = data + text_from_html(html)

In [23]:
data

'                                                                                Doorport wants to make your apartment building’s front door smarter     Greg Kumparak   @grg  / 9 hours         If you live in a big city, you’ve probably had your fair share of battles with apartment intercom systems. Those electronic gatekeepers with their tiny screens, sticky buttons, and seemingly endless lists of names to tap through in search of a friend who can buzz you in.  Doorport wants to make those existing systems a bit smarter. They’ve built a device that can be wired into existing buzzer systems, allowing you to use your smartphone to unlock your building’s door for yourself and your guests with a quick tap. Once installed, the existing intercom system works just as it did before — just now with a bit more smarts.  The company’s current prototype hardware is about the size of a deck of cards, and is meant to be tucked into the empty space within an already in-place intercom system. The compa

In [9]:
"""
for i in range(0,2): #'a' tags are for links #len(soup.findAll('a'))+1
    one_a_tag = soup.findAll('a')[i]
    link = one_a_tag['href']
    urllib.request.urlretrieve(link)
    time.sleep(1) #pause the code for a sec, aviod spamming the website with requests
"""

"\nfor i in range(0,2): #'a' tags are for links #len(soup.findAll('a'))+1\n    one_a_tag = soup.findAll('a')[i]\n    link = one_a_tag['href']\n    urllib.request.urlretrieve(link)\n    time.sleep(1) #pause the code for a sec, aviod spamming the website with requests\n"

##Set Up Google Cloud Natural Language API
documentation: [Google APIs & Python libraries](https://cloud.google.com/python/docs/reference/)

In [10]:
import io
import os
from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types
from google.colab import files
uploaded = files.upload() #Somwhow mounting google drive and access json file does not work with cloud language API (works perfectly for vision API), so I had to upload the file to colab
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="BAX452 practice-ccb307020798.json"
client = language.LanguageServiceClient.from_service_account_json("BAX452 practice-ccb307020798.json") # Instantiates a client


##Perform NLP Analysis using Cloud Natual Language
[Cloud Natural Language API documentation](https://cloud.google.com/natural-language/docs/)

###Sentiment Analysis

In [11]:
#Sentiment analysis using Cloud Natural Language
document = language.types.Document(
     content= data,
     type='PLAIN_TEXT',
 )
response = client.analyze_sentiment(
     document=document,
     encoding_type='UTF32',
 )
sentiment = response.document_sentiment
print(sentiment.score)

print(sentiment.magnitude)

0.0
17.100000381469727


###Entity Analysis

In [12]:
import six

text = data

if isinstance(text, six.binary_type):
    text = text.decode('utf-8')

# Instantiates a plain text document.
document = types.Document(
    content=text,
    type=enums.Document.Type.PLAIN_TEXT)

# Detects entities in the document. You can also analyze HTML with:
#   document.type == enums.Document.Type.HTML
entities = client.analyze_entities(document).entities

for entity in entities:
    entity_type = enums.Entity.Type(entity.type)
    print('=' * 20)
    print(u'{:<16}: {}'.format('name', entity.name))
    print(u'{:<16}: {}'.format('type', entity_type.name))
    print(u'{:<16}: {}'.format('salience', entity.salience))
    print(u'{:<16}: {}'.format('wikipedia_url',
          entity.metadata.get('wikipedia_url', '-')))
    print(u'{:<16}: {}'.format('mid', entity.metadata.get('mid', '-')))

name            : robots
type            : OTHER
salience        : 0.10851119458675385
wikipedia_url   : -
mid             : -
name            : robots
type            : OTHER
salience        : 0.07959903031587601
wikipedia_url   : -
mid             : -
name            : Amazon
type            : ORGANIZATION
salience        : 0.07781699299812317
wikipedia_url   : https://en.wikipedia.org/wiki/Amazon_(company)
mid             : /m/0mgkg
name            : battle
type            : EVENT
salience        : 0.042608730494976044
wikipedia_url   : -
mid             : -
name            : process
type            : OTHER
salience        : 0.03690604865550995
wikipedia_url   : -
mid             : -
name            : JFK8
type            : OTHER
salience        : 0.02816653624176979
wikipedia_url   : -
mid             : /g/11g234l5lx
name            : fulfillment center
type            : ORGANIZATION
salience        : 0.02754194661974907
wikipedia_url   : -
mid             : -
name            : pac

## Set AWS Comprehend API
[AWS documentation](https://docs.aws.amazon.com/ses/latest/DeveloperGuide/create-shared-credentials-file.html)

In [13]:
#Mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
os.listdir("/content/gdrive/My Drive/.aws")

Mounted at /content/gdrive


['config', 'credentials']

In [0]:
#install boto
!pip -q install boto3
import boto3

In [0]:
#Create API Config
!mkdir -p ~/.aws
!cp /content/gdrive/My\ Drive/.aws/credentials ~/.aws/credentials
comprehend = boto3.client(service_name='comprehend', region_name="us-east-1")

###Test Comprehend API Call


In [16]:
text = "There is smoke in San Francisco"
comprehend.detect_sentiment(Text=text, LanguageCode='en')

{'ResponseMetadata': {'HTTPHeaders': {'content-length': '163',
   'content-type': 'application/x-amz-json-1.1',
   'date': 'Sun, 17 Mar 2019 20:59:17 GMT',
   'x-amzn-requestid': '87a77064-48f7-11e9-89c1-1bb0e911a101'},
  'HTTPStatusCode': 200,
  'RequestId': '87a77064-48f7-11e9-89c1-1bb0e911a101',
  'RetryAttempts': 0},
 'Sentiment': 'NEUTRAL',
 'SentimentScore': {'Mixed': 0.0032753823325037956,
  'Negative': 0.05816865712404251,
  'Neutral': 0.929866373538971,
  'Positive': 0.008689661510288715}}

##Perform NLP Analysis using AWS Comprehend

###Sentiment Analysis

In [17]:
import json
json.dumps(comprehend.detect_sentiment(Text=data, LanguageCode='en'), sort_keys=True, indent=4)

TextSizeLimitExceededException: ignored

###Detect Syntax

In [0]:
comprehend = boto3.client(service_name='comprehend', region_name='region')
text = data
 
print('Calling DetectSyntax')
print(json.dumps(comprehend.detect_syntax(Text=text, LanguageCode='en'), sort_keys=True, indent=4))
print('End of DetectSyntax\n')